In [1]:
# Install necessary libraries
#!pip install geopandas shapely pyproj fiona ipywidgets plotly --quiet

In [2]:
# -----------------------------
# Imports: Bibliotheken:

## GeoPandas ist ein Open-Source-Projekt, das die Arbeit mit Geodaten in Python vereinfachen soll. Es erweitert von Pandas verwendeten Datentypen, um räumliche Operationen auf geometrischen Typen zu ermöglichen.
## Shapely: Ein Python-Paket zur Manipulation und Analyse planarer geometrischer Objekte. `box` wird verwendet, um Rechtecke zu erzeugen, und `Polygon` wird verwendet, um Hexagone zu erzeugen.
## ipywidgets: Ein Python-Paket zur Erstellung interaktiver Benutzeroberflächen.
## matplotlib: Ein Python-Paket zur Visualisierung von Ergebnissen und Karten.
# -----------------------------
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box, Polygon
import math
from ipywidgets import FileUpload, Dropdown, Button, Output, RadioButtons, VBox, FloatSlider
from IPython.display import display, clear_output


class CreateTileMap:

    def __init__(self):
        # -----------------------------
        # Interaktive Widgets
        # -----------------------------
        self.upload = FileUpload(accept='.geojson', multiple=False)  # Datei-Upload für GeoJSON
        self.label_selc = Dropdown(description='Beschriftung:',style={'description_width': '50px'})  # Die Spalte für Beschriftung
        #self.tile_shape_selc = RadioButtons(options=['Rechteck', 'Hexagon'], description='Tile Shape:')  # Rechteck oder Hexagon
        
        self.tile_shape_selc = RadioButtons(options=['Rectangle', 'Hexagon'], description='Tile Shape:')  # Rechteck oder Hexagon

        self.tile_scaling_factor = FloatSlider(value=0.98, min=0.50, max=2, step=0.01, description='Scaling Factor:',style={'description_width': '100px'})  # Skalierungsfaktor zur Anpassung der Kachelgröße
        self.generate_button = Button(description='Generate Tiles', button_style='success')  # Startknopf
        self.export_button = Button(description='Export GeoJSON', button_style='info')  # Exportknopf
        self.output = Output()  # Bereich für Ausgaben
        
        
        # -----------------------------
        # Globale Variablen
        # -----------------------------
        self.polygons = None ## Hier wird der GeoPandas DataFrame aus der Originaldatei gespeichert    
        self.snapped_grid = None ## Hier werden die zugewiesenen Kacheln gespeichert  
        self.final_grid = None ## Dies ist eine Kopie von self.snapped_grid, aber mit allen anderen Attributen aus der Originaldatei, um das Resultat für den Export vorzubereiten.
        self.output_filename = "output_tiles.geojson" ## Initialname als Platzhalter  
        self.base_name = None ## Hier wird der Hauptname aus der Originaldatei ohne die Erweiterung .geojson extrahiert.
        
        
        ### Hier wird die Upload-Funktion verwendet, um die Originaldatei zu lesen. 
        self.upload.observe(self.handle_upload, names='value')
        # -----------------------------
        # Button-Aktionen verbinden
        # -----------------------------
        self.generate_button.on_click(self.generate_tiles)
        self.export_button.on_click(self.export_geojson)

        # -----------------------------
        # Hier wird die Benutzeroberfläche angezeigt. VBox wird verwendet.
        ## Die VBox-Helferklasse bietet einfache Standardeinstellungen, um Child-Widgets in vertikalen Boxen anzuordnen.
        # -----------------------------
        
        display(VBox([
            self.upload,
            self.label_selc,
            self.tile_shape_selc,
            self.tile_scaling_factor,
            self.generate_button,
            self.export_button,
            self.output
        ]))

           
    # -----------------------------
    # Upload-Handler
    # Lädt GeoJSON-Datei, liest sie und konvertiert sie in  EPSG:3857 (Web Mercator)
    # Fügt eine UID-Spalte (eindeutige ID) hinzu.
    # Listet alle Spalten aus der Originaldatei in einem Dropdown-Menü in der UI zur Beschriftung auf.
    # -----------------------------
    def handle_upload(self,change):
        if self.upload.value:
            for filename, fileinfo in self.upload.value.items():
                with open(filename, 'wb') as f:
                    f.write(fileinfo['content'])

                self.base_name = str(filename).replace(".geojson","")
                self.polygons = gpd.read_file(filename)
                self.polygons = self.polygons.to_crs(epsg=3857)
                self.polygons["UID"] = self.polygons.index.astype(str)
                self.label_selc.options = list(self.polygons.columns)
                
                
    # -----------------------------
    # # Hilfsfunktion: Berechnet die optimale Kachelgröße anhand von Fläche und Zielanzahl
    # -----------------------------
 
    #Mathematische Logik:
    #Ziel: Die Fläche der Bounding Box soll in target_count gleich große Zellen unterteilt werden.

    #Die Gesamtfläche:
    #Fläche = (maxx - minx) * (maxy - miny) (basierend auf dem Originaldatensatz)

    #Fläche der Zielzellen:
    #Zielzellenfläche= Fläche / target_count (target_count entspricht in der Regel der Anzahl der ursprünglichen Polygone)

    #Zellseitenlänge:
    #sqrt(Zielzellenfläche), da die Zellen quadratisch sind (d.h. die Basisgröße)

    #In diesem Schritt wird ein geeigneter Wert für die Zellseitenlänge berechnet, 
    #so dass die Gesamtzahl der generierten Kacheln in etwa der Anzahl der Polygone aus dem Originaldatensatz entspricht. 
    #Dieser Ansatz führt jedoch häufig zu Lücken und einer ungleichmäßigen Verteilung der Zellen.
    #Aufgrund der Überschneidung der generierten Kacheln mit der Gesamtfläche aller Polygone im Originaldatensatz 
    #ist die Anzahl der überlappenden Kacheln oft geringer, was eine eindeutige 1:1-Zuordnung erschwert.
    #Aus diesem Grund wurde später die Idee der Verwendung eines Skalierungsfaktors eingeführt. 
    #Dieser Skalierungsfaktor ermöglicht eine gezielte Anpassung der berechneten Zellengröße. 
    #Auf diese Weise können etwas mehr Kacheln erzeugt werden, als ursprünglich Polygone vorhanden waren. 
    #Das Ergebnis ist eine dichtere Kachelkarte, bei der die Zellen näher beieinander liegen, 
    #wodurch eine bessere räumliche Abdeckung erreicht und eine optimale 1:1-Zuordnung ermöglicht wird.
    # -----------------------------

    def calculate_optimal_tile_size(self,bounds, target_count):
        minx, miny, maxx, maxy = bounds
        area = (maxx - minx) * (maxy - miny)
        tile_area = area / target_count
        size = math.sqrt(tile_area)
        return size, size
    
    
    
    # -----------------------------
    # Hauptfunktion: Erzeugt das Kachelgitter und ordnet die Kacheln den Polygonen zu.
    # Der Parameter b ist ein Platzhalter für das Button-Widget, das das Ereignis ausgelöst hat.
    # -----------------------------
    def generate_tiles(self,b):
        with self.output:
            clear_output(wait=True)
            self.export_button.disabled = True

            if self.polygons is None:
                print("Bitte zuerst eine GeoJSON-Datei hochladen.")
                return

            name_label = self.label_selc.value
            tile_shape = self.tile_shape_selc.value

            ##Berechnet die Flächenschwerpunkte aller Polygone
            self.polygons["centroid"] = self.polygons.geometry.centroid
            minx, miny, maxx, maxy = self.polygons.total_bounds
            target_cells = len(self.polygons)
            
            ##Initial oder theoretische Standardgröße der Zellen, ohne Anpassung der Skalierung
            grid_width, grid_height = self.calculate_optimal_tile_size(self.polygons.total_bounds, target_cells)
            grid_size = self.tile_scaling_factor.value # Vom Benutzer gewählter Skalierungsfaktor

            grid = []
            max_attempts = 15
            attempt = 0

            # -----------------------------
            # Wiederhole die Erzeugung, bis die Anzahl der Kacheln >= Anzahl der Polygone ist.
            # Iterative Größenanpassung:
            # Ein Schieberegler steuert die Größe der Kacheln anhand eines Skalierungsfaktors. Das Skript passt diesen Wert iterativ an -
            # aber nur, wenn die Optimierung weniger als 15 Iterationen erfordert, um Rechenzeit zu sparen.
            # Damit wird sichergestellt, dass eine ausreichende Anzahl von Kacheln nach dem Intersect das Zielgebiet abdecken kann (>= Anzahl der Polygone).
            # Je näher die Gesamtzahl der erzeugten Kacheln nach dem Intersect an der Anzahl der ursprünglichen Polygone liegt,
            # desto besser und übersichtlicher wird das Endergebnis sein.
            # -----------------------------
            
            print("Gesamtzahl der Polygone: ",len(self.polygons) )
            print("Optimale grid_height vor Anwendung des Skalierungsfaktors: ",grid_height)
            print(9*"#")
            print()
            
            while len(grid) != len(self.polygons) and attempt < max_attempts:
                grid_cells = []
                
                # -----------------------------
                # Rechteckige Kacheln generieren:
                # In diesem Abschnitt wird ein rechteckiges Gitter erzeugt, indem das gesamte Begrenzungsfeld systematisch mit Kacheln gefüllt wird.
                # Für jede Position wird ein Rechteck (Kachel) mit fester Breite und Höhe (grid_width × grid_height) erzeugt.
                # Die verschachtelten Schleifen iterieren über die X- und Y-Koordinaten, um alle Zellen nahtlos und lückenlos zu erzeugen.
                # -----------------------------

                #if tile_shape == "Rechteck":

                if tile_shape == "Rectangle":
                    x = minx
                    while x < maxx:
                        y = miny
                        while y < maxy:
                            grid_cells.append(box(x, y, x + grid_width, y + grid_height))
                            y += grid_height
                        x += grid_width


                        
                # -----------------------------
                # Hexagonale Kacheln generieren

                # 1. Größenberechnung:
                # hex_height: Höhe eines Hexagons - definiert durch die gewählte grid_height.
                # hex_width: Breite eines Hexagons - abgeleitet von der Geometrie eines regelmäßigen Hexagons:
                # hex_width = (sqrt(3) / 2) * hex_height

                # 2. Gitterkonstruktion - Vertikal (y-Achse):
                # Beginn bei miny, iterier Zeile für Zeile bis maxy + hex_height.
                # Jede neue Zeile wird um hex_height * 0,75 nach unten verschoben.
                # So entsteht ein nahtloses Wabenmuster.
                # Der Faktor 0,75 berücksichtigt die vertikale Überlappung zwischen benachbarten Reihen.

                # 3. Rasteraufbau - Horizontal (x-Achse):
                # In jeder zweiten Reihe (ungerade Reihennummer) wird der Startpunkt horizontal
                # um eine halbe Hexagonbreite verschoben: (row % 2) * (hex_width / 2).
                # So entsteht die gestaffelte Wabenanordnung.

                # 4. Hexagon generieren:
                # Die Ecken des Hexagons werden mit Hilfe von Polarkoordinaten berechnet:
                # 6 Winkel im Abstand von 60° (von -30° bis 300°), so dass das Hexagon flach liegt.
                # Jeder Punkt wird berechnet als:
                # x_i = x + r * cos(θ), y_i = y + r * sin(θ)
                # wobei r = hex_height / 2.
                # Diese Punkte bilden ein regelmäßiges flaches Hexagon,
                # das dann als Polygon-Objekt gespeichert wird.

                # 5. Hexagon speichern:
                # Jedes erzeugte hexagonale Polygon wird an die Liste grid_cells angehängt.
                # -----------------------------

                elif tile_shape == "Hexagon":
                    hex_height = grid_height
                    hex_width = math.sqrt(3) / 2 * hex_height
                    y = miny
                    row = 0
                    while y < maxy + hex_height:
                        x = minx - (row % 2) * (hex_width / 2)
                        while x < maxx + hex_width:
                            angles = [math.radians(a) for a in range(-30, 330, 60)]
                            points = [(x + (hex_height / 2) * math.cos(a), y + (hex_height / 2) * math.sin(a)) for a in angles]
                            grid_cells.append(Polygon(points))
                            x += hex_width
                        y += hex_height * 0.75  
                        row += 1
                else:
                    raise ValueError("Ungültiger tile_shape")

                    
                # -----------------------------  
                # Erstelle ein GeoDataFrame aus den generierten Kacheln
                # Filtert alle Kacheln heraus, die sich nicht mit der Zielregion schneiden (Vereinigung aller Polygone).
                # Berechne dann den Schwerpunkt jeder verbleibenden Kachel für die spätere Zuordnung
                # -----------------------------
                grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs=self.polygons.crs)
                print("Anzahl der Kacheln vor dem Intersect: ",len(grid))
                region_union = self.polygons.unary_union
                grid = grid[grid.intersects(region_union)].reset_index(drop=True)
                grid["grid_centroid"] = grid.geometry.centroid
                print("Anzahl der Kacheln nach dem Intersect: ",len(grid))
                
            
            
                if len(grid) >= len(self.polygons):
                    print(9*"#")
                    print()
                    print(f"Die Kachelgröße wurde nach {attempt} Iterationen erfolgreich angepasst.")
                    print()
                    break  
                else:
                    grid_width *= grid_size
                    grid_height *= grid_size
                    print(9*"#")
                    print()
                    print("Optimale grid_height nach Anwendung des Skalierungsfaktors: ",grid_height)
                attempt += 1
                    
                if attempt == max_attempts:
                    print(f"Abbruch nach {max_attempts} Versuchen. Kachelanzahl: {len(grid)} / Polygone: {len(self.polygons)}")
                    if len(grid) < len(self.polygons):
                        print("Wählen Sie einen niedrigen Wert aus...")
                        
                    return None

                
            # ---------------------------------
            # Jedes Polygon wird der Kachel mit dem nächstgelegenen Flächenschwerpunkt zugeordnet.
            ## Snapping-Algorithmus:
            ## Der Schwerpunkt jedes Eingabepolygons wird der nächstgelegenen verfügbaren Gitterzelle zugewiesen.
            ## Sobald eine Zelle zugewiesen ist, wird sie aus dem Pool entfernt (greedy approach).
            ### Für jede Zeile (Polygon) im Datensatz wird der Flächenschwerpunkt berechnet.
            # Der Algorithmus findet die nächstgelegene Gitterzelle, indem er die Abstände zwischen dem Polygonschwerpunkt und allen Gitterzellenschwerpunkten berechnet.
            # Die nächstgelegene Rasterzelle wird als „zugewiesen“ markiert und aus den verfügbaren Rasterzellen entfernt.
            # um doppelte Zuweisungen zu vermeiden.
            # Die zugewiesenen Rasterzellen und ihre entsprechenden ursprünglichen Polygonbezeichnungen werden gesammelt.
            # Ein neuer GeoDataFrame mit dem Namen `snapped_grid` wird dann erstellt,
            # der die gefangenen Rasterzellen zusammen mit den Beschriftungen enthält.
            # Außerdem werden zusätzliche Attribute aus den ursprünglichen Polygonen in `final_grid` übertragen.
            # -----------------------------

            assigned_geoms = []
            assigned_labels = []
            available_grids = grid.copy()

            for idx, row in self.polygons.iterrows():
                centroid = row["centroid"]
                available_grids["distance"] = available_grids["grid_centroid"].distance(centroid)
                candidate_idx = available_grids["distance"].idxmin()
                assigned_geoms.append(available_grids.loc[candidate_idx].geometry)
                assigned_labels.append(row[name_label])
                available_grids = available_grids.drop(index=candidate_idx)

            self.snapped_grid = gpd.GeoDataFrame({
                "geometry": assigned_geoms,
                name_label: assigned_labels
            }, crs=self.polygons.crs)

            # Übertrage die restlichen Attribute aus den Originaldaten.
            self.final_grid = self.snapped_grid.copy()
            try:
                self.final_grid = self.snapped_grid.drop(columns=[name_label]).copy()
            except:
                pass

            attr = self.polygons.drop(columns=['geometry', 'centroid']).reset_index(drop=True)
            self.final_grid = self.final_grid.join(attr)
            tile_size_str = f"{int(grid_width)}x{int(grid_height)}"
            crsText=str(self.polygons.crs).replace(":","_")
            self.output_filename = f"{self.base_name}_Tilemap_{tile_size_str}_{tile_shape}_{crsText}.geojson"
            self.export_button.disabled = False
            print("#### Kacheln erfolgreich generiert: ")
            print("Gesamtzahl der Polygone:", len(self.polygons))
            print("Gesamtzahl der Kacheln:", len(grid))
            print("Zugeordnete Kacheln:", len(self.snapped_grid))

            
            
            
            # -----------------------------
            # Das Resultat Plotten
            #Erstellt eine Grafik, die die Umrisse der „snapped_grid“ (in Blau) und der Originalpolygone 
            #(in Schwarz, transparent) darstellt. Für jede Kachel in „snapped_grid“ wird das geometrische 
            #Zentrum berechnet und an dieser Position ein Textlabel mit einem gekürzten Attributwert angezeigt. 
            #Der Plot erhält den Titel „Zugeordnete Kacheln“, nutzt ein gleichmäßiges Seitenverhältnis und optimiert 
            #das Layout vor der Anzeige.
            # --------------------------------
            
            fig, ax = plt.subplots(figsize=(12, 8))
            self.snapped_grid.boundary.plot(ax=ax, edgecolor='blue', linewidth=0.8, alpha=0.7)
            self.polygons.boundary.plot(ax=ax, edgecolor='black', alpha=0.3)
            for idx, row in self.snapped_grid.iterrows():
                centroid = row.geometry.centroid
                ax.text(centroid.x, centroid.y, str(row[name_label])[:10], fontsize=5, ha='center', color='darkred')
            ax.set_title("Zugeordnete Kacheln")
            ax.set_aspect('equal')
            plt.tight_layout()
            plt.show()
            
    # ------------------------------------
    # Exportfunktion: Speichert die finale Tilemap als GeoJSON EPSG:3857
    # Der Parameter b ist ein Platzhalter für das Button-Widget, das das Ereignis ausgelöst hat.
    # -----------------------------------
    def export_geojson(self, b):
        if self.final_grid is None:
            with self.output:
                print("Keine Tilemap vorhanden. Bitte zuerst 'Generate Tiles' klicken.")
            return

        self.final_grid.to_file(self.output_filename, driver='GeoJSON')
        with self.output:
            print(f"Datei exportiert: {self.output_filename}")



#===usage ===
if __name__ == "__main__":
    obj = CreateTileMap()